# 动态规划
- 暴力递归为什么暴力是因为有着很多的重复运算，我们可以采用缓存的方式来记录这些重复计算的内容，使得下一次遇见相同递归的时候可以直接调用。(记忆化搜索)
-  关键点在于可变参数有多少，我们就创建多大的缓存表，再根据规则可以得到最终结果。




## 问题1
假设有排成一行N个位置，记为1到N，N一定>=2。开始时机器人在M位置上(M满足1 <= M <= N)，机器人行走规则为：
- 如果机器人来到1位置那么下一步只能来到2位置；
- 如果机器人来到N位置下一步只能来到N-1位置；
- 如果机器人来到中间位置，那么下一步可以往左一步也可以往右一步，

问K步后，机器人到达P位置(P满足1 <= P <= N)，有多少种走法。给定四个参数N，M，K，P返回有多少种方法数。

分析：
- 记忆化搜索：用一个字典储存(M,K)的数量（即从M点到K有多少种不同路线），如果之后再遇见则可以直接提取。
- 在使用递归计算的时候产生重复计算，因此可以用一个二维列表(注意不能用\[\[-1]\*5]*3来生成矩阵，这种方式无法对指定位置更改数据。)来记录下不同的递归返回值，当再次遇见相同的递归返回值时,可以直接调用。(记忆化搜索)
- 可以根据递归的分析过程抽象出动态规划的转移方程。
- 如果需要记录最终的路径则不能采用记忆化搜索提前储存路径，并且要注意还原现场

In [ ]:
### 不储存路径的方式(记忆化搜索)
dic = dict() # 用于缓存不同的M,K取得的种类数，若存在已经走过的路径直接提取线路数量
def process(N, M, K, P):

    # 如果计算结果已经在缓存里了，则直接调取
    if (M,K) in dic:
        return dic[(M, K)]

    # 剩余步数为0且当前位置为P，则记录该次结果
    if K == 0 and M == P:
        dic[(M, K)] = 1
        return dic[(M, K)]

    if K == 0 and M != P:
        dic[(M, K)] = 0
        return dic[(M, K)]

    if M == 1:
        dic[(M, K)] = process(N, 2, K-1, P)
        return dic[(M, K)]

    if M == N:
        dic[(M, K)] = process(N, N-1, K-1, P)
        return dic[(M, K)]

    count = 0
    # 计算左移
    count = count + process(N, M-1, K-1, P) 
    
    # 计算右移
    count = count + process(N, M+1, K-1, P)
    dic[(M, K)] = count
    return dic[(M, K)]

process(4, 2, 3, 1)

In [ ]:
# K表示还有K步，M表示当前位置移动到的位置，P表示目标位置,N表示一共有多少位置,返回种类数
arr = []    # 记录当前方案
all = []    # 记录所有方案
dic = dict() # 用于缓存不同的M,K取得的种类数，若存在已经走过的路径直接提取线路数量

def process(N, M, K, P):
    # 剩余步数为0且当前位置为P，则记录该次结果
    if K == 0 and M == P:
        all.append(arr.copy())
        return 1
    if K == 0 and M != P:
        return 0

    if M == 1:
        arr.append(2)  # 把2放入当前方案
        count = process(N, 2, K-1, P)
        arr.pop() # 还原现场
        return count
    if M == N:
        arr.append(N-1)
        count = process(N, N-1, K-1, P)
        arr.pop()
        return count

    count = 0
    # 计算左移
    arr.append(M-1)
    count = count + process(N, M-1, K-1, P) 
    arr.pop()
    
    # 计算右移
    arr.append(M+1)
    count = count + process(N, M+1, K-1, P)
    arr.pop()
    return count

print(process(4, 2, 3, 1))
print(all)

## 问题2(背包问题)
有两个长度均为N的数组,每个位置分别代表该件产品weight和value。现在你有一个背包，可容纳重量为bag，请问在不超过容纳重量的前提下，装下货物的最大价值为多少?

改为动态规划:
- 仅根据递归来更改为动态规划，采用二维表的方式来计算，形成4行10列的矩阵(字典)可以采用以下三种方式
```python
dict([(i, [0]*10) for i in range(4)])
{i:[0]*10 for i in range(4)}
[[i for i in range(10)] for j in range(4)]
```
- 先做出能装下所有情况的二维表
- 动态规划的更改其实就是在一般情况中
- dp[i][rest]表示：到第i件商品为止，剩余rest重量，能获得的最大价值。
- 注意dp[i]和weight[i]下标含义不同。

In [ ]:
import numpy as np
weights = [20, 5, 8, 4, 11]
values = [17, 8, 14, 45, 12]
bag = 40

# rest指剩余的重量
def dpway(weights, values, bag):
    N = len(weights)
    dp = np.zeros((N+1, bag+1)) #生成N+1行bag+1列的矩阵
    # dp =  dict([(i,[0]*(bag+1)) for i in range(N+1)])#生成N+1行bag+1列的矩阵

    for i in range(1, N+1): 
        for rest in range(1, bag+1):
            # 不拿当前物品
            maxValue2 = dp[i-1][rest] 
            # 假设拿当前物品
            if rest >= weights[i-1]:  # 这里要注意i的符号
                maxValue1 = values[i-1] + dp[i-1][rest-weights[i-1]]
            else:
                maxValue1 = 0 # 如果当前重量超过了剩余，则拿不了当前的价值
            # 最后取最大值
            dp[i][rest] = max(maxValue1, maxValue2)

    return dp[N][bag]

dpway(weights, values, bag)

## 问题3(字符串转化)
规定1与A对应，2与B对应，3与C对应...那么一个数字字符串比如“111”可以转化为“AAA”、“KA”、“AK”。问给定一个数字字符串，有多少种转化方法？

递归方法参看《12暴力递归》问题6

改为动态规划：
- 本题是从末尾往前计算。dp[i]表示从第i位置到N-1位置为止解码的方法数


In [ ]:
arr = "9211234126"
def dpway(arr):
    N = len(arr)
    dp = [0]*(N+1)
    dp[N] = 1

    for i in range(N-1, -1, -1):
        if arr[i] == "0":   # 如果数字是0也没办法转化“01，02是不合规的”
            dp[i] = 0
        elif arr[i] == "1":
            count = dp[i+1]
            if i+2 <= len(arr): # 保证末尾有2个数字
                count = count + dp[i+2]
            dp[i] = count
        elif arr[i] == "2": 
            count = dp[i+1]
            if i+2 <= len(arr) and int(arr[i:i+2]) <= 26: 
                count = count + dp[i+2]
            dp[i] = count
        else:
            dp[i] = dp[i+1]
    return dp[0]

dpway(arr)

## 问题4 (范围上尝试模型)
给定一个整型数组arr，代表数值不同的纸牌排成一条线，玩家A和玩家B每次拿走一张纸牌，规定玩家A先拿，玩家B后拿，但是每个玩家每次只能拿走最左边或者最右边的纸牌，玩家A和玩家B都绝顶聪明。请返回最后获胜者的分数

更改为动态规划：
- 互相嵌套的动态规划.
- first是正方形表，second也是正方形表
- 范围上的尝试有一半位置是没有的，因为L>R是无效的
- 相互调用，F的值依赖于S的值，S的值依赖于F的值，所以我们需要交替更新。
- 更新对角线采用while循环，每次都行列++，由于每次F和S的更新都是不是依赖于上一对角线所以可以同步进行。放在一个循环里。
- F[i][j]表示从arr数组的【i，j】范围内先手拿的最大得分，S[i][j]表示从arr数组的【i，j】后手拿的最大得分。
- 初始化：从【k，k】位置先手拿的最大分数就是arr[k]，后手拿的分数就是0。
- 转移方程：F[L][R]: max(拿左边+后手拿接下来的数字，拿右边+后手拿接下来的数字)；S[L][R]: 需要根据F[L][R]的选择才能得到最后答案，如果F[L][R]选择拿左边，则S[L][R]=F[L+1][R]...（下面代码中有更简洁的写法，但是我不知道为什么）

In [ ]:
arr = [20, 5, 8, 4, 11]
def dpway(arr):
    N = len(arr)
    F = dict([(i, [0]*(N)) for i in range(N)])
    S = dict([(i, [0]*(N)) for i in range(N)])

    for i in range(N):
        F[i][i] = arr[i]
        S[i][i] = 0

    for R in range(1, N):
        L = 0
        while(R < N):
            # ### 简单理解版本
            # if arr[L] + S[L+1][R] > arr[R] + S[L][R-1]:
            #     F[L][R] = arr[L] + S[L+1][R]
            #     S[L][R] = F[L+1][R]
            # else:
            #     F[L][R] = arr[R] + S[L][R-1]
            #     S[L][R] = F[L][R-1]
            # ### 代码简单版本
            F[L][R] = max(arr[L] + S[L+1][R],
                          arr[R] + S[L][R-1])
            S[L][R] = min(F[L+1][R], F[L][R-1])
            L = L + 1
            R = R + 1
    print(F)
    print(S)
    return max(F[0][N-1], S[0][N-1])
dpway(arr)

## 问题5(完全背包问题)
给定一个数组arr,每个位置表示面值,且无重复面值,每个可以使用无限次.给定一个指定面额，问用给定数组有多少种方法可以得到该面额？

### 方法一(递归方法)
- 每一张都是用无限次(遍历)，直到rest<0
- 遍历完了就是用下一张。

In [ ]:
arr = [3,10,4,19,2,12,5]
rest = 12
all = []    # 记录所有面值张数的组合
ans = []    # 记录每次的组合
# index：从index到最后的面额都可以使用
# rest：还需要rest的
# 返回：表示的方法数
def process(arr, index, rest, ans):
    if index == len(arr):
        if rest == 0:
            all.append(ans.copy())
            return 1
        else:
            return 0

    count = 0   # 用于计数
    zhang = 0   # 这一面额的钱的使用数量
    while zhang * arr[index] <= rest :
        ans.append(zhang)
        count += process(arr, index+1, rest-zhang * arr[index], ans)
        ans.pop()
        zhang += 1
        
    return count
print(process(arr, 0, rest, ans))
all

### 方法二(记忆搜索的方法)
- 递归过程总子过程出现了多次重复的情况，因此我们可以利用一个字典来记录子过程的返回值(index,rest):value


In [ ]:
arr = [3,10,4,19,2,12,5]
rest = 12
dp = dict() # 用来作为缓存，(index, rest)从index位置表示rest的钱有多少种方式。

def ways(arr, index, rest):
    if index == len(arr):
        if rest == 0:
            dp[(index, rest)] = 1
            return dp[(index, rest)]
        else:
            dp[(index, rest)] = 0
            return dp[(index, rest)]

    # 如果在记录里则直接返回递归结果即可
    if (index, rest) in dp:
        return dp[(index, rest)]

    count = 0   #用于计数
    zhang = 0   #这一面额的钱的使用数量
    while zhang * arr[index] <= rest :
        count += ways(arr, index+1, rest-zhang * arr[index])
        zhang += 1
    dp[(index, rest)] = count

    return dp[(index, rest)]

ways(arr, 0, rest)

### 方法三(动态规划方法)
- 准备一张二维数组，dp\[index]\[rest] = 次数
- 我们可以填出最后一行
- 发现每一行的值都只和下一行有关(某些指定为位置的数值)
- 发现规律:每一个数据其实可以表示为当前前第x个格子的数值和当前格子下一层的数值有关
- dp[index][rest]: 表示从index到最后，表示rest的种类数。
- dp[i][0] = 1 ，对任意i

In [56]:
arr = [3,10,4,19,2,12,5]
rest = 12

def ways(arr, rest):
    N = len(arr)
    dp = dict([(i,[0]*(rest+1)) for i in range(N+1)])  #此处的维度必须得每个位置+1

    dp[N][0] = 1 
    for index in range(N-1, -1,-1):
        for j in range(rest+1):
            if j >= arr[index]:
                # 不要当前数值的情况 + 再要一个当前数值的情况
                dp[index][j] = dp[index+1][j] + dp[index][j-arr[index]]
            else:
                dp[index][j] = dp[index+1][j]
    print(dp[N-3])
    return dp[0][rest]
ways(arr, rest)

[1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 2, 1, 3]


12